In [ ]:
!pip install ipywidgets

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

## Setting Constants and Field Conditions:

In [2]:
#Defining constants
Mb = 145e-3 #standardized mass of mlb baseball in kg
g = 9.8 #m/s^2
Rb = 74e-3/2 #standardized radius of a mlb baseball in meters
A = np.pi * Rb**2 #cross sectional area of a baseball

Cd = 0.3 #Drag coefficient on a baseball
rho = 1.23 #kg/m^3

#initializing baseball pitch conditions (distance to plate, initial throw height)
relHeight=1.8 #m, about 6 feet
relExtension=1.8 #same value, very good exension down the mound

## Setting initial conditions for pitch:

In [3]:
x0=0     # Definition of initial x position.

y0=relHeight    # Definition of initial y position.

z0=0          #Definition of initial z position

## Creating time array for our pitch modeling:

In [4]:
t0=0                            # Definition of initial time.

tf=2                            # Definition of final time.

N=100                         # Definition of slice number N.

h=(tf-t0)/N                     # Definition of slice width h.

timeArray=np.linspace(t0,tf,N)  # Definition of our time array using t0, tf, and N.

## Defining Function that inputs initial conditions of spin and trajectory and outputs forces to use in rk45 method:

In [5]:
def flightPath(trajectory,spin):                             # Definition of our differential equation function.
    '''
    This function takes three-dimensional trajectory and spin arrays as inputs and uses three dimensional 
    equations of motion to calculate the changing position and velocity of the baseball through time. 

    Parameters:

        trajectory: A 1x6 array containing instantaneous position and velocity of the baseball, each in all three
                    dimensions.

        spin: A 1x3 array containing instantaneous spin of the baseball about all three dimensional axes.

    Return:

        np.array([dxdt,dydt,dzdt,dvxdt,dvydt,dvzdt],float): A new 1x6 array containing instantaneous three-dimensional 
                                                            positon and velocity values of the ball that are generated from
                                                            the input position and velocity values being modified by the
                                                            equations of motion that include acceleration due to gravity, drag,
                                                            and the Magnus effect. 

    '''
    x=trajectory[0]                                # Extraction of x component from the trajectory array.
    
    spinx=spin[0]                                  #Extracting spin about x axis
    
    y=trajectory[1]                                # Extraction of y component from the trajectory array.
    
    spiny=spin[1]                                  # Extracting spin about y axis
    
    z=trajectory[2]                                # Extraction of z component from the trajectory array.
    
    spinz=spin[2]                                  #Extracting spin about z axis 

    
    vx=trajectory[3]                               # Extraction of vx component from the argument array.

    vy=trajectory[4]                               # Extraction of vy component from the argument array.

    vz=trajectory[5]                               # Extraction of vy component from the argument array.

    
    dxdt=vx                                        # Redefinition of vx.

    dydt=vy                                        # Redefinition of vy.

    dzdt=vz                                        # Redefinition of vz.
    
    vTot=trajectory[3:]
    vMagnitude=np.sqrt(np.dot(vTot,vTot))
    stot = abs(spinx) + abs(spiny) + abs(spinz)
    
    
    #adding in if statement to prevent function from erroring while calculating forces in the case of no spin:
    if stot == 0:
        dvxdt=(-Cd*A*rho/Mb)*vMagnitude*vx     # Definition of dvxdt.

        dvydt=-g-(Cd*A*rho/Mb)*vMagnitude*vy   # Definition of dvydt.

        dvzdt=(-Cd*A*rho/Mb)*vMagnitude*vz

    #forces implimented in the magnus force
    else:
       #Definition of necessary factors for magnus acceleration
        spinMagnitude=np.sqrt(np.dot(spin,spin))
        

        #Defining the Acceleration in each dimension:
        dvxdt=(-Cd*A*rho/Mb)*np.sqrt(vx**2+vy**2+vz**2)*vx+(1/Mb)*0.5*(0.62*(Rb*spinMagnitude/vMagnitude)**0.7)*rho*A*vMagnitude/spinMagnitude*np.cross(spin,trajectory[3:])[0]

        dvydt=-g-(Cd*A*rho/Mb)*np.sqrt(vx**2+vy**2+vz**2)*vy+(1/Mb)*0.5*(0.62*(Rb*spinMagnitude/vMagnitude)**0.7)*rho*A*vMagnitude/spinMagnitude*np.cross(spin,trajectory[3:])[1]   

        dvzdt=(-Cd*A*rho/Mb)*np.sqrt(vx**2+vy**2+vz**2)*vz+(1/Mb)*0.5*(0.62*(Rb*spinMagnitude/vMagnitude)**0.7)*rho*A*vMagnitude/spinMagnitude*np.cross(spin,trajectory[3:])[2]

    return np.array([dxdt,dydt,dzdt,dvxdt,dvydt,dvzdt],float) # Return of an array containing information necessary to
                                                   # define changes in velocity over time.


## Defining function that takes our trajectory and spin arrays as inputs and outputs a modeled trajectory array in 3 dimensions:

In [6]:
def rk45(trajectory, spin):
    for j in range(N-1):                                             # rk45 for loop that iterates over all rows of the trajectory array.

        trajectoryCur=trajectory[j,:]                           # Definition of the current values to be worked with via extraction from the current
                                                                 # trajectoryArray row.

        k1=h*flightPath(trajectoryCur,spin)                                    # Definition of rk45 variables
        k2=h*flightPath(trajectoryCur+k1/2,spin)                               # using the current values from above
        k3=h*flightPath(trajectoryCur+k2/2,spin)                               # as the argument of our differential equation
        k4=h*flightPath(trajectoryCur+k3,spin)                                 # function, along with slice width h.

        trajectory[j+1,:]=trajectoryCur+(k1+2*k2+2*k3+k4)/6     # Definition of the next row of our trajectoryArray using the freshly defined
                                                                 # rk45 variables.

    xVals=trajectory[:,0]              # Extraction of our x values from the finished trajectoryArray.

    yVals=trajectory[:,1]              # Extraction of our y values from the finished trajectoryArray.

    zVals=trajectory[:,2]              # Extraction of our z values from the finished trajectoryArray.

    vxVals=trajectory[:,3]             # Extraction of our vx, vy, and vz values from the finished trajectoryArray, just for good measure.
    vyVals=trajectory[:,4]             # We will not plot these, but will make them accessible just to have the full picture available.
    vzVals=trajectory[:,5]
    positiveYVals=[]                                 # Initialization of our new limited x values array.

    positiveXVals=[]                                 # Initialization of our new limited y values array.

    positiveZVals=[]                                 # Initialization of our new limited z values array. 


    #This loop exists to cut off the trajectory calculation if the ball hits the ground (y=0), or if the ball reaches the plate (x=19)
    for j in range(len(yVals)):                      # For loop that will iterate over the length of the original y values array.
        if xVals[j]<=(18.89-relExtension):                              # If the y value is positive, the y value and the corresponding x value will be
            if yVals[j]>=0:
                positiveYVals.append(yVals[j])           # added to each respective new limited value arrays.
                positiveXVals.append(xVals[j])
                positiveZVals.append(zVals[j])
        else:
                break
    return positiveXVals, positiveYVals, positiveZVals

## Defining the three dimensional home plate and batter:

In [19]:
# Definition of the x, y, and z arrays of the position of the home plate and the batter. 
homePlateWidth=0.432 # Width of home plate in meters.

strikeZoneBottom=0.457 # Typical lowest point of the strike zone, m. 

strikeZoneTop=1.067 # Typical highest point of the strike zone, m.

# Definition of the strike zone positional arrays using the constants defined above.
strikeZoneZArray=np.array([-homePlateWidth/2,homePlateWidth/2,homePlateWidth/2,-homePlateWidth/2,-homePlateWidth/2])

strikeZoneY=np.array([strikeZoneBottom,strikeZoneBottom,strikeZoneTop,strikeZoneTop,strikeZoneBottom])

strikeZoneXVal=(18.189-relExtension) # Definition of the x distance from the pitcher's release point to home plate using the previously defined
                                     # relExtension and the distance from the pitcher's mound to home plate, about 60 ft, 6 inches in meters (18.189). 
strikeZoneXArray=np.array([strikeZoneXVal,strikeZoneXVal,strikeZoneXVal,strikeZoneXVal,strikeZoneXVal])

# Definition of the batter positional arrays for a batter sillhouette that is 0.4 meters wide and 1.8 meters tall.
batterXArray=strikeZoneXArray
batterZArray=np.array([0.4,0.8,0.8,0.4,0.4])
batterY=np.array([0,0,1.8,1.8,0])

## All parts compiled into 2D RK45 Plotting Function:

In [20]:
def master2DFunction(xAxisSpin,yAxisSpin,zAxisSpin):
    """
    This function takes three spin input arrays (one for each dimension) and creates a 2-dimensional plot of the final position of the spin-modified 
    pitch and no-spin control pitch in the z-y plane along with 2-dimensional plots of the strike zone and batter. 

    Parameters:

        xAxisSpin: The component of spin of the baseball about the x axis.

        yAxisSpin: The component of spin of the basbeall about the y axis.

        zAxisSpin: The component of spin of the basbeall about the y axis.

    Returns:

        Plot: The function does not have any traditional returns, but instead has the explicit purpose of producing the 2-dimensional plot
               of the strike zone, batter, and final locations of the two pitches. Essentially, the function returns this plot. 
    """
    # Conversion of the separately user-input initial velocity of the ball to a float with units in m/s.
    velocity=float(v0)
    velocity=0.447*velocity

    # Conversion of the separately use-input initial vertical and horizontal trajectory angles to floats in units of radians. 
    verticalAngle= (np.pi/180) * float(theta0)
    horizontalAngle = -float(phi0) + 90
    horizontalAngle = (np.pi/180) * horizontalAngle

    # Conversion of the three arguments to float values. 
    xAxisSpin=float(xAxisSpin)
    yAxisSpin=-float(yAxisSpin)
    zAxisSpin=float(zAxisSpin)

    # Usage of the three arguments to create a three-dimensional spin array in units of rad/s and definition of our no spin, spin array.  
    spinArrayComp=0.105*np.array([xAxisSpin,yAxisSpin,zAxisSpin])
    noSpinArrayComp=np.array([0,0,0])
    
    # Definitions of the initial rows of the spin-impacted and no-spin trajectory arrays and initialization of said arrays with those rows assigned
    # to the corresponding arrays. An empty base array is copied to make the process a bit more efficient. The two arrays, spin and no spin, look identical
    # in their first rows, as they both have the same initial conditions except for the spin. Each trajectory array has the format [x0,y0,z0,vx0,xy0,vz0].
    trajectoryInit0=np.array([x0,y0,z0,velocity*np.cos(verticalAngle)*np.sin(horizontalAngle),velocity*np.sin(verticalAngle),velocity*np.cos(verticalAngle)*np.cos(horizontalAngle)],float)
    trajectoryInit_base=np.empty([N,6]) 
    trajectoryArrayInit=trajectoryInit_base.copy()
    trajectoryArrayInit[0,:]=trajectory0
    trajectoryArrayInitNoSpin=trajectoryInit_base.copy()
    trajectoryArrayInitNoSpin[0,:]=trajectory0

    # Implementation of the rk45 functions for the spin and no spin trajectories using the trajectory and spin arrays defined above for each case. 
    xSpinMF,ySpinMF,zSpinMF=rk45(trajectoryArrayInit,spinArrayComp)
    xNoSpinMF,yNoSpinMF,zNoSpinMF=rk45(trajectoryArrayInitNoSpin,noSpinArrayComp)

    
    # Definition of our 2-d figure and plotting of the final positional values of the spin and no spin trajectory arrays calculated above using rk45.
    plt.figure(figsize = (homePlateWidth*10,(strikeZoneTop-strikeZoneBottom)*10))
    #plt.scatter(nsfinal, sfinal, color = 'r')
    plt.plot(zSpinMF[-1],ySpinMF[-1],'w.',mec='b',label="Pitch With Spin",markersize=20)
    plt.plot(zNoSpinMF[-1],yNoSpinMF[-1],'w.',mec='y',label="Pitch With No Spin",markersize=20)


    plt.xlim(-1,1)
    plt.ylim(0,2)
    plt.title('Strike Zone with Batter')
    plt.xlabel('z(t)')
    plt.ylabel('y(t)')
    plt.grid('major')                   
    #strikeZoneX=np.array([-homePlateWidth/2,homePlateWidth/2,homePlateWidth/2,-homePlateWidth/2,-homePlateWidth/2])
    strikeZoneY=np.array([strikeZoneBottom,strikeZoneBottom,strikeZoneTop,strikeZoneTop,strikeZoneBottom])
    plt.plot(strikeZoneZArray,strikeZoneY,'g', label = 'Strike Zone')
    batterX=np.array([0.4,0.8,0.8,0.4,0.4])
    batterY=np.array([0,0,1.8,1.8,0])
    plt.plot(batterX,batterY,'k', label = 'Batter', linestyle = 'dashed')
    plt.legend()


In [9]:
def master2DFunctionSide(xAxisSpin,yAxisSpin,zAxisSpin):
    velocity=float(v0)
    velocity=0.447*velocity
    
    verticalAngle= (np.pi/180) * float(theta0)
    horizontalAngle = -float(phi0) + 90
    horizontalAngle = (np.pi/180) * horizontalAngle
    
    xAxisSpin=float(xAxisSpin)
    yAxisSpin=-float(yAxisSpin)
    zAxisSpin=float(zAxisSpin)
    
    spinArrayComp=0.105*np.array([xAxisSpin,yAxisSpin,zAxisSpin])
    noSpinArrayComp=np.array([0,0,0])
    
    #Variable definitions and rk45 implementation.
    
    trajectoryInit0=np.array([x0,y0,z0,velocity*np.cos(verticalAngle)*np.sin(horizontalAngle),velocity*np.sin(verticalAngle),velocity*np.cos(verticalAngle)*np.cos(horizontalAngle)],float)
    trajectoryInit_base=np.empty([N,6]) 
    trajectoryArrayInit=trajectoryInit_base.copy()
    trajectoryArrayInit[0,:]=trajectory0
    trajectoryArrayInitNoSpin=trajectoryInit_base.copy()
    trajectoryArrayInitNoSpin[0,:]=trajectory0
    
    xSpinMF,ySpinMF,zSpinMF=rk45(trajectoryArrayInit,spinArrayComp)
    xNoSpinMF,yNoSpinMF,zNoSpinMF=rk45(trajectoryArrayInitNoSpin,noSpinArrayComp)
    
    #Plot generation
    
    plt.figure(figsize = (10,3))
    plt.plot(xNoSpinMF, yNoSpinMF,'y.',label="No Spin",markersize=10)
    plt.plot(xSpinMF, ySpinMF, 'b', label="With User-Defined Spin", linestyle = 'dashed')
    plt.title('Side View of Baseball Trajectory With and Without Spin')
    plt.xlabel("x Position, Pitcher's Release Point to Plate")
    plt.ylabel("Height of Ball Above Ground")
    plt.xlim(0, 18)
    plt.ylim(0, 2)
    plt.legend()
    plt.grid()

In [10]:
def master2DFunctionBirdsEye(xAxisSpin,yAxisSpin,zAxisSpin):
    velocity=float(v0)
    velocity=0.447*velocity
    
    verticalAngle= (np.pi/180) * float(theta0)
    horizontalAngle = -float(phi0) + 90
    horizontalAngle = (np.pi/180) * horizontalAngle
    
    xAxisSpin=float(xAxisSpin)
    yAxisSpin=-float(yAxisSpin)
    zAxisSpin=float(zAxisSpin)
    
    spinArrayComp=0.105*np.array([xAxisSpin,yAxisSpin,zAxisSpin])
    noSpinArrayComp=np.array([0,0,0])
    
    #Variable definitions and rk45 implementation.
    
    trajectoryInit0=np.array([x0,y0,z0,velocity*np.cos(verticalAngle)*np.sin(horizontalAngle),velocity*np.sin(verticalAngle),velocity*np.cos(verticalAngle)*np.cos(horizontalAngle)],float)
    trajectoryInit_base=np.empty([N,6]) 
    trajectoryArrayInit=trajectoryInit_base.copy()
    trajectoryArrayInit[0,:]=trajectory0
    trajectoryArrayInitNoSpin=trajectoryInit_base.copy()
    trajectoryArrayInitNoSpin[0,:]=trajectory0
    
    xSpinMF,ySpinMF,zSpinMF=rk45(trajectoryArrayInit,spinArrayComp)
    xNoSpinMF,yNoSpinMF,zNoSpinMF=rk45(trajectoryArrayInitNoSpin,noSpinArrayComp)
    
    plt.figure(figsize = (10,3))
    plt.plot(xNoSpinMF, zNoSpinMF,'y.',label="No Spin",markersize=10)
    plt.plot(xSpinMF, zSpinMF, 'b',label="With User-Defined Spin",linestyle="dashed")
    plt.title('Birds Eye View of Baseball Trajectory')
    plt.xlabel('x(t)')
    plt.ylabel('z(t)')
    plt.xlim(0, 18)
    plt.ylim(1, -1)
    plt.legend()
    plt.grid()
    
    
    

## All parts compiled 3D RK45 Plotting Function:

In [11]:
def master3DFunction(xAxisSpin,yAxisSpin,zAxisSpin):
    
    #Value recalibrations:
    velocity=float(v0)
    velocity=0.447*velocity
    
    verticalAngle= (np.pi/180) * float(theta0)
    horizontalAngle = -float(phi0) + 90
    horizontalAngle = (np.pi/180) * horizontalAngle
    
    xAxisSpin=float(xAxisSpin)
    yAxisSpin=-float(yAxisSpin)
    zAxisSpin=float(zAxisSpin)
    
    spinArrayComp=0.105*np.array([xAxisSpin,yAxisSpin,zAxisSpin])
    noSpinArrayComp=np.array([0,0,0])
    
    #Variable definitions and rk45 implementation.
    
    trajectoryInit0=np.array([x0,y0,z0,velocity*np.cos(verticalAngle)*np.sin(horizontalAngle),velocity*np.sin(verticalAngle),velocity*np.cos(verticalAngle)*np.cos(horizontalAngle)],float)
    trajectoryInit_base=np.empty([N,6]) 
    trajectoryArrayInit=trajectoryInit_base.copy()
    trajectoryArrayInit[0,:]=trajectory0
    trajectoryArrayInitNoSpin=trajectoryInit_base.copy()
    trajectoryArrayInitNoSpin[0,:]=trajectory0
    
    xSpinMF,ySpinMF,zSpinMF=rk45(trajectoryArrayInit,spinArrayComp)
    xNoSpinMF,yNoSpinMF,zNoSpinMF=rk45(trajectoryArrayInitNoSpin,noSpinArrayComp)
    
    #3d plotting
    fig=plt.figure(figsize = (10, 10))
    ax=fig.add_subplot(111,projection='3d',xlim=(-1,1),ylim=(0,18),zlim=(0,2))
    ax.set_xlabel("z(t)m")
    ax.set_ylabel("x(t)m")
    ax.set_zlabel("y(t)m)")
    ax.plot(zSpinMF,xSpinMF,ySpinMF,'b')
    ax.plot(zNoSpinMF,xNoSpinMF,yNoSpinMF,'y')
    ax.plot(strikeZoneZArray,strikeZoneXArray,strikeZoneY,'g')
    ax.plot(batterZArray,batterXArray,batterY,'k')
    ax.set_box_aspect([1, 4, 1])  # [x, y, z] aspect ratio
    #ax.set_ylim(0, 18)
    #ax.set_zlim(0, 2)
    #ax.set_xlim(-1, 1)
    ax.set_xticks([-1, 0, 1])
    ax.set_zticks([0, 1, 2])
    ax.plot(zSpinMF[-1],xSpinMF[-1],ySpinMF[-1],'w.',mec='b',label="Pitch With Spin",markersize=15)
    ax.plot(zNoSpinMF[-1],xNoSpinMF[-1],yNoSpinMF[-1],'w.',mec='y',label="Pitch With No Spin",markersize=15)
    ax.set_title('3d View of pitch with User input initial conditions')
    ax.view_init(10, -60)
    ax.legend()
    plt.show()

# Input Initial Conditoins: 

## Input Initial velocity

In [12]:
#input initial speed
v0= input('Enter your pitch speed in mph:')
v0 = float(v0)
v0 = 0.447 * v0
                 # Definition of the first row of trajectoryArray using trajectory0.

Enter your pitch speed in mph: 95


## Input Initial Trajectory Angles:

In [13]:
##### input initial angle
theta0= input('Input the initial vertical angle of the pitch in degrees (positive angle sends pitch up, negative sends pitch down): ')
theta0 = (np.pi/180) * float(theta0)
phi0 = input('Input the initial horizontal angle of the pitch in degrees (positive angle sends pitch to the right of the plate, negative to the left): ')
phi0 = -float(phi0) + 90
phi0 = (np.pi/180) * phi0

Input the initial vertical angle of the pitch in degrees (positive angle sends pitch up, negative sends pitch down):  0
Input the initial horizontal angle of the pitch in degrees (positive angle sends pitch to the right of the plate, negative to the left):  1


# Input Initial Spin:

In [14]:
#input spin per axes
spinx0 = input('Input the spin on the baseball about the x axis (rpm): ') #user input for x, y, and z axes respectively
spinx0 = float(spinx0) #floating the inputs for the respective axes
spiny0 = input('Input the spin on the baseball about the y axis (rpm): ')
spiny0 = -float(spiny0)
spinz0 = input('Input the spin on the baseball about the z axis (rpm): ')
spinz0 = float(spinz0)

spinArray = 0.105*np.array([spinx0, spiny0, spinz0]) #Factorization of rpm to rad/s

Input the spin on the baseball about the x axis (rpm):  0
Input the spin on the baseball about the y axis (rpm):  0
Input the spin on the baseball about the z axis (rpm):  0


# Side View Plot:

In [15]:
%matplotlib inline
%matplotlib notebook

from ipywidgets import interact, FloatSlider

interact(master2DFunctionSide, xAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinx0),\
        yAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spiny0),zAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinz0))

interactive(children=(FloatSlider(value=0.0, description='xAxisSpin', max=4000.0, min=-4000.0, step=100.0), Fl…

<function __main__.master2DFunctionSide(xAxisSpin, yAxisSpin, zAxisSpin)>

# Bird's Eye View Plot:

In [16]:
interact(master2DFunctionBirdsEye, xAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinx0),\
        yAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spiny0),zAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinz0))

interactive(children=(FloatSlider(value=0.0, description='xAxisSpin', max=4000.0, min=-4000.0, step=100.0), Fl…

<function __main__.master2DFunctionBirdsEye(xAxisSpin, yAxisSpin, zAxisSpin)>

# Strike Zone Plot:

In [17]:
interact(master2DFunction, xAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinx0),\
        yAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spiny0),zAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinz0))

interactive(children=(FloatSlider(value=0.0, description='xAxisSpin', max=4000.0, min=-4000.0, step=100.0), Fl…

<function __main__.master2DFunction(xAxisSpin, yAxisSpin, zAxisSpin)>

# 3 Dimensional Interactive Plot

In [18]:
interact(master3DFunction, xAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinx0),\
        yAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spiny0),zAxisSpin=FloatSlider(min=-4000,max=4000,step=100,value=spinz0))

interactive(children=(FloatSlider(value=0.0, description='xAxisSpin', max=4000.0, min=-4000.0, step=100.0), Fl…

<function __main__.master3DFunction(xAxisSpin, yAxisSpin, zAxisSpin)>